# Awaybot
Kevin Davis, Walter Erquinigo, Guillermo Monge, Carlos Rodriguez & Alex Smith

***
### Resources
- <a href="https://www.fullstackpython.com/blog/build-first-slack-bot-python.html">How to Build Your First Slack Bot with Python</a>
- <a href="https://medium.com/@julianmartinez/how-to-write-a-slack-bot-with-python-code-examples-4ed354407b98#.t8q6p76kj">How To Write a Slack Bot — with Python Code Examples</a>
- <a href="http://angus.readthedocs.io/en/2014/amazon/transfer-files-between-instance.html">Transfer Files Between Your Laptop and an EC2 Instance</a>

***
Add the file from your local computer to the EC2 instance:<br>
```
scp -i /Users/Alex/Documents/Berkeley/1603Fall/W210/Architecture/Capstone.pem /Users/Alex/Documents/Berkeley/1603Fall/W210/Slackbot/awaybot.py root@ec2-54-175-51-154.compute-1.amazonaws.com:/usr/lib/
```

***
### Test out the bot to ensure it's working all good

In [2]:
%%writefile print_bot_id.py
import os
from slackclient import SlackClient


BOT_NAME = 'awaybot_test_aks'

slack_client = SlackClient(os.environ.get('SLACK_BOT_TOKEN'))


if __name__ == "__main__":
    api_call = slack_client.api_call("users.list")
    if api_call.get('ok'):
        # retrieve all users so we can find our bot
        users = api_call.get('members')
        for user in users:
            if 'name' in user and user.get('name') == BOT_NAME:
                print("Bot ID for '" + user['name'] + "' is " + user.get('id'))
    else:
        print("could not find bot user with the name " + BOT_NAME)

Writing print_bot_id.py


In [4]:
!python print_bot_id.py

Bot ID for 'awaybot_test_aks' is U2JFJ5ETU


In [9]:
# set the bot_id as a global environment variable
!export BOT_ID='U2JFJ5ETU'

### Awaybot code

In [1]:
%%writefile awaybot.py
import os
import time
from slackclient import SlackClient

# awaybot's ID as an environment variable
BOT_ID = os.environ.get("BOT_ID")

# constants
AT_BOT = "<@" + 'U2JFJ5ETU' + ">"
COMMAND = "/summarize"

# instantiate Slack & Twilio clients
slack_client = SlackClient(os.environ.get('SLACK_BOT_TOKEN'))

def handle_command(command, channel):
    """
        Receives commands directed at the bot and determines if they
        are valid commands. If so, then acts on the commands. If not,
        returns back what it needs for clarification.
    """
    
    # response for when the bot doesn't know quite what to do
    response = """Not sure what you mean. Use the */summarize* command with the *channel name* and the *duration*. For example, if you want to see 3 weeks of history in the #general channel, type: */summarize #general 3 weeks*"""
        
    # if the command is a known command, modify the response to
    # the correct response
    if command.startswith(COMMAND):
        response = "Sure...write some more code then I can do that!"
    
    # post the response to the channel
    slack_client.api_call("chat.postMessage", channel=channel,
                          text=response, as_user=True)


def parse_slack_output(slack_rtm_output):
    """
        The Slack Real Time Messaging API is an events firehose.
        this parsing function returns None unless a message is
        directed at the Bot, based on its ID.
    """
    
    # by default set the output to what was spit out 
    # by the fire hose
    output_list = slack_rtm_output
    
    # if we actually got something and not just an
    # empty array
    if output_list and len(output_list) > 0:
        
        # loop through each of the messages
        for output in output_list:
            
            # if we have some text in the message and 
            # this bot is mentioned
            if output and 'text' in output and AT_BOT in output['text']:
                
                # return text after the @ mention, whitespace removed
                return output['text'].split(AT_BOT)[1].strip().lower(), \
                       output['channel']
    return None, None



if __name__ == "__main__":
    
    # set a 1 second delay between reading from firehose
    READ_WEBSOCKET_DELAY = 1
    
    # try to connect to the real time messaging API
    if slack_client.rtm_connect():
        print("Awaybot connected and running!")
        
        # keep a continuous loop
        while True:
            
            # parse the output from the client using our
            # pre-defined parser function
            command, channel = parse_slack_output(slack_client.rtm_read())
            
            # if we have both a command and a channel, then
            # use our predefined handle command function
            if command and channel:
                handle_command(command, channel)
                
            # sleep for the number of seconds before reading from
            # the fire hose again
            time.sleep(READ_WEBSOCKET_DELAY)
    
    # if something went wrong, print out what happened
    else:
        print("Connection failed. Invalid Slack token or bot ID?")

Overwriting awaybot.py


In [ ]:
# start up the away bot
!python awaybot.py

Awaybot connected and running!
